### Bibliotecas Úteis 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from seaborn import color_palette
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, recall_score

%matplotlib inline

### Leitura dos Dados

In [2]:
dados = pd.read_csv('dengue_tp_fix.csv',low_memory=False)
print(dados.shape)
dados.head()

(97831, 33)


,tp_sexo,tp_gestante,tp_raca_cor,tp_escolaridade,co_distrito_residencia,co_bairro_residencia,tp_classificacao_final,tp_evolucao_caso,febre,mialgia,...,hematolog,hepatopat,renal,hipertensao,acido_pept,auto_imune,notificao_dias,idade,caso_internacao,train
0,M,6.0,9.0,10.0,122.0,820.0,5,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,334.0,2.0,SIM,T
1,F,9.0,9.0,9.0,119.0,841.0,5,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,365.0,12.0,SIM,T
2,M,6.0,4.0,NaN,118.0,826.0,5,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,122.0,22.0,SIM,T
3,F,6.0,9.0,10.0,122.0,820.0,1,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,61.0,4.0,SIM,T
4,F,9.0,9.0,NaN,122.0,824.0,5,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,61.0,46.0,SIM,T


In [3]:
dados.columns

Index(['tp_sexo', 'tp_gestante', 'tp_raca_cor', 'tp_escolaridade',
       'co_distrito_residencia', 'co_bairro_residencia',
       'tp_classificacao_final', 'tp_evolucao_caso', 'febre', 'mialgia',
       'cefaleia', 'exantema', 'vomito', 'nausea', 'dor_costas', 'conjutivite',
       'artrite', 'artralgia', 'petequia_n', 'leucopenia', 'laco', 'dor_retro',
       'diabetes', 'hematolog', 'hepatopat', 'renal', 'hipertensao',
       'acido_pept', 'auto_imune', 'notificao_dias', 'idade',
       'caso_internacao', 'train'],
      dtype='object')

### Divisão dos subsets - Treino e Teste

In [4]:
treino = dados[dados.train=='T'].drop('train',axis=1)
print(treino.shape)
print(treino.caso_internacao.value_counts())
teste = dados[dados.train!='T'].drop('train',axis=1)
print(teste.shape)
print(teste.caso_internacao.value_counts())

(68920, 32)
SIM    34460
NAO    34460
Name: caso_internacao, dtype: int64
(28911, 32)
NAO    14879
SIM    14030
Name: caso_internacao, dtype: int64


### Regressão Logística

In [5]:
treinoCP = sm.add_constant(treino.copy())
treinoCP['Ip'] = treinoCP.caso_internacao.map(lambda x: x=='SIM' and 1 or 0)
model = smf.logit("Ip ~  + tp_sexo + tp_gestante + tp_escolaridade + tp_classificacao_final + notificao_dias + idade + tp_raca_cor + tp_escolaridade + co_distrito_residencia + co_bairro_residencia + febre + mialgia + cefaleia + exantema + vomito + nausea + dor_costas + conjutivite + artrite + artralgia + petequia_n + leucopenia + laco + dor_retro + diabetes + hematolog + hepatopat + renal + hipertensao + acido_pept + auto_imune",data=treinoCP).fit()

model.summary()

Optimization terminated successfully.
         Current function value: 0.264870
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     Ip   No. Observations:                50747
Model:                          Logit   Df Residuals:                    50715
Method:                           MLE   Df Model:                           31
Date:                Mon, 24 Jun 2019   Pseudo R-squ.:                  0.6121
Time:                        18:33:20   Log-Likelihood:                -13441.
converged:                       True   LL-Null:                       -34652.
                                        LLR p-value:                     0.000
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 38.7697      4.258      9.105      0.000      30.424      47.115
tp_sexo[T.I]              -0.2067      0.364     -0.568      0.570      -0.920       0.507
tp_sexo[T.M]              -0.0816      0.033     -2.496      0.013      -0.146      -0.018
tp_gestante               -0.3245      0.011    -28.847      0.000      -0.347      -0.302
tp_escolaridade            0.0181      0.011      1.594      0.111      -0.004       0.040
tp_classificacao_final    -0.1160      0.007    -16.642      0.000      -0.130      -0.102
notificao_dias            -0.0009      0.000     -2.564      0.010      -0.002      -0.000
idade                      0.0311      0.001     39.535      0.000       0.030       0.033
tp_raca_cor               -0.0242      0.007     -3.653      0.000      -0.037      -0.011
co_distrito_residencia     0.0031      0.000     26.789      0.000       0.003       0.003
co_bairro_residencia       0.0097      0.000     44.086      0.000       0.009       0.010
febre                     -3.0090      0.062    -48.388      0.000      -3.131      -2.887
mialgia                   -1.2907      0.064    -20.024      0.000      -1.417      -1.164
cefaleia                  -0.9174      0.065    -14.012      0.000      -1.046      -0.789
exantema                  -0.2289      0.076     -3.007      0.003      -0.378      -0.080
vomito                    -1.6377      0.095    -17.165      0.000      -1.825      -1.451
nausea                     0.0569      0.096      0.594      0.552      -0.131       0.245
dor_costas                -1.3137      0.225     -5.852      0.000      -1.754      -0.874
conjutivite               -1.6134      0.424     -3.807      0.000      -2.444      -0.783
artrite                    0.3255      0.176      1.847      0.065      -0.020       0.671
artralgia                  0.5958      0.071      8.347      0.000       0.456       0.736
petequia_n                -1.9907      0.270     -7.380      0.000      -2.519      -1.462
leucopenia                -2.4086      0.350     -6.887      0.000      -3.094      -1.723
laco                      -1.2778      0.630     -2.028      0.043      -2.513      -0.043
dor_retro                  0.5157      0.103      5.013      0.000       0.314       0.717
diabetes                  -1.6974      0.644     -2.634      0.008      -2.961      -0.434
hematolog                  0.5296      0.661      0.801      0.423      -0.766       1.826
hepatopat                 -0.0790      0.717     -0.110      0.912      -1.484       1.326
renal                     -0.5118      1.130     -0.453      0.651      -2.727       1.703
hipertensao               -3.1107      0.505     -6.160      0.000      -4.100      -2.121
acido_pept                -2.8907      1.057     -2.736      0.006      -4.962      -0.820
auto_imune                 2.2729      0.695      3.270      0.001       0.911       3.635
========================================================

### Predição de Valores

<strong>Probabilidade</strong> de ser um Caso de internação

In [6]:
model.predict(teste)

40308         NaN
40309    0.978109
40310         NaN
40311    0.986592
40312    0.988932
40313    0.853750
40314    0.989833
40315    0.288750
40316    0.780430
40317    0.409276
40318         NaN
40319         NaN
40320         NaN
40321    0.754969
40322         NaN
40323    0.990621
40324    0.970507
40325         NaN
40326         NaN
40327         NaN
40328         NaN
40329    0.625777
40330    0.062336
40331    0.584992
40332         NaN
40333    0.215617
40334    0.070842
40335    0.965707
40336    0.978167
40337         NaN
           ...   
97801    0.997528
97802    0.999264
97803         NaN
97804    0.994324
97805         NaN
97806    0.994871
97807    0.874152
97808    0.998116
97809    0.987572
97810         NaN
97811         NaN
97812    0.106736
97813    0.997357
97814    0.966404
97815    0.607916
97816    0.736728
97817    0.999068
97818    0.344212
97819         NaN
97820    0.984942
97821         NaN
97822    0.996290
97823    0.075661
97824    0.994576
97825    0

Como podemos ver, o modelo retorna as <strong>probabilidades associadas</strong> a cada objeto no conjunto de teste de <strong>terem sido internados!</strong>
Podemos converter tais probabilidades em classes. Podemos arbitrariamente dizer que se a probabilidade for <strong>maior que 0.5</strong> então a classe é <strong>sim para internação</strong>, e <strong>negativa caso contrário.</strong>

In [7]:
model.predict(teste).map(lambda x: x > 0.5 and 'SIM' or 'NAO')

40308    NAO
40309    SIM
40310    NAO
40311    SIM
40312    SIM
40313    SIM
40314    SIM
40315    NAO
40316    SIM
40317    NAO
40318    NAO
40319    NAO
40320    NAO
40321    SIM
40322    NAO
40323    SIM
40324    SIM
40325    NAO
40326    NAO
40327    NAO
40328    NAO
40329    SIM
40330    NAO
40331    SIM
40332    NAO
40333    NAO
40334    NAO
40335    SIM
40336    SIM
40337    NAO
        ... 
97801    SIM
97802    SIM
97803    NAO
97804    SIM
97805    NAO
97806    SIM
97807    SIM
97808    SIM
97809    SIM
97810    NAO
97811    NAO
97812    NAO
97813    SIM
97814    SIM
97815    SIM
97816    SIM
97817    SIM
97818    NAO
97819    NAO
97820    SIM
97821    NAO
97822    SIM
97823    NAO
97824    SIM
97825    SIM
97826    SIM
97827    SIM
97828    SIM
97829    NAO
97830    NAO
Length: 28911, dtype: object

### Comparando os valores originais com a predição


In [37]:
df = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.7 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [38]:
df

,prob,pred,real
40308,NaN,NAO,NAO
40309,0.978109,SIM,NAO
40310,NaN,NAO,NAO
40311,0.986592,SIM,NAO
40312,0.988932,SIM,NAO
40313,0.853750,SIM,NAO
40314,0.989833,SIM,NAO
40315,0.288750,NAO,NAO
40316,0.780430,SIM,NAO
40317,0.409276,NAO,NAO


Podemos ainda sumarizar os resultados numa tabela de contigência, mostrando os acertos e erros do modelo.

In [39]:
pd.crosstab(df.pred,df.real)

real,NAO,SIM
pred,,
NAO,10615,4254
SIM,4264,9776


Essa tabela é conhecida como tabela de contingência. A diagonal principal mostra os acertos do modelo, enquanto a secundária mostra os erros. Os exemplos que eram realmente maus e foram classificados como tal são chamados de verdadeiros positivos (TP), já os bons classificados corretamente são chamados de verdadeiros negativos (TN). Os bons classificados de maus são chamados de falsos positivos (FP) e os maus classificados de bons são falsos negativos.

A acurácia do modelo é a proporção de acertos que o modelo teve. Além dessa métrica, outras duas métricas de qualidade são sensitividade e especificidade que revelam a capacidade do modelo de classificar corretamente os exemplos que eram positivos (maus) e os que eram negativos (bons).



Dessa forma, podemos avaliar a qualidade do nosso modelo.

<strong>MODELO BASE</strong>

In [40]:
tabcont = pd.crosstab(df.pred,df.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.705, sen=0.697, esp=0.713


Assim observamos que nosso <strong>modelo acertou 70%</strong> das classificações, sendo mais preciso com exemplos positivos <strong>(casos de internação), acertando 69%</strong>, que com exemplos negativos <strong>(casos de não-internação), acertando 71%.</strong>

## Outros modelos
Como queremos acertar mais os casos de internação, vamos usar taxas de probabilidade de aceitação menores para ampliarmos o alcance do modelo aos casos de internação perdidos.

<strong>MODELO 1</strong>

In [41]:
modelo1 = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.65 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [42]:
pd.crosstab(modelo1.pred,modelo1.real)

real,NAO,SIM
pred,,
NAO,10433,4126
SIM,4446,9904


In [43]:
tabcont = pd.crosstab(modelo1.pred,modelo1.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.703, sen=0.706, esp=0.701


<strong>MODELO 2</strong>

In [46]:
modelo2 = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.6 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [47]:
pd.crosstab(modelo2.pred,modelo2.real)

real,NAO,SIM
pred,,
NAO,10228,4038
SIM,4651,9992


In [48]:
tabcont = pd.crosstab(modelo2.pred,modelo2.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.699, sen=0.712, esp=0.687


<strong>MODELO 3</strong>

In [49]:
modelo3 = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.5 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [50]:
pd.crosstab(modelo3.pred,modelo3.real)

real,NAO,SIM
pred,,
NAO,9855,3861
SIM,5024,10169


In [51]:
tabcont = pd.crosstab(modelo3.pred,modelo3.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.693, sen=0.725, esp=0.662


<strong>MODELO 4</strong>

In [52]:
modelo4 = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.45 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [53]:
pd.crosstab(modelo4.pred,modelo4.real)

real,NAO,SIM
pred,,
NAO,9682,3733
SIM,5197,10297


In [54]:
tabcont = pd.crosstab(modelo4.pred,modelo4.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.691, sen=0.734, esp=0.651


<strong>MODELO 5</strong>

In [55]:
modelo5 = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.3 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [56]:
pd.crosstab(modelo5.pred,modelo5.real)

real,NAO,SIM
pred,,
NAO,8829,3211
SIM,6050,10819


In [57]:
tabcont = pd.crosstab(modelo5.pred,modelo5.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.680, sen=0.771, esp=0.593


<strong>MODELO 6</strong>

In [58]:
modelo6 = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.25 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [59]:
pd.crosstab(modelo6.pred,modelo6.real)

real,NAO,SIM
pred,,
NAO,8443,2977
SIM,6436,11053


In [60]:
tabcont = pd.crosstab(modelo6.pred,modelo6.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.674, sen=0.788, esp=0.567


<strong>MODELO 7</strong>

In [61]:
modelo7 = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.2 and 'SIM' or 'NAO'),
                  real=teste.caso_internacao))

In [62]:
pd.crosstab(modelo7.pred,modelo7.real)

real,NAO,SIM
pred,,
NAO,7937,2789
SIM,6942,11241


In [63]:
tabcont = pd.crosstab(modelo7.pred,modelo7.real).values
acc = np.sum(tabcont.diagonal())/teste.shape[0]
sen = tabcont[1,1]/(tabcont[1,1]+tabcont[0,1])
esp = tabcont[0,0]/(tabcont[0,0]+tabcont[1,0])

print("acc= {:.3f}, sen={:.3f}, esp={:.3f}".format(acc,sen,esp))

acc= 0.663, sen=0.801, esp=0.533


## Conclusão - modelos

<strong>Analisando os modelos criados podemos concluir que a taxa de aceitação, usada para classificar como casos de internação ou não, quanto menor for maior sera o acerto nos casos que precisam serem internados porem ha uma grande perda na acuracia dos casos que não precisam serem internados. Assim ajustamos a taxa de classificação de acordo com o objetivo que precisamos alcançar!</strong>